<a href="https://colab.research.google.com/github/FarhanDhanani/joker-clef-22-FAST-MT/blob/main/JOKER_CLEF_TASK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> This notebook is prepared to submit the solutions to the JOKER CLEF conference from the FAST-MT Team.</center></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/en/e/e4/National_University_of_Computer_and_Emerging_Sciences_logo.png"/></center>

A simple set of booleans that represents the notebook is running on google colab or locally. And the model are already downloaded and loaded either on G-Drive or locally or it needs to be downloaded from the checkpoint repository.

In [1]:
'''
## A set of simple booleeans:
##  - (run_on_colab) = True : if notebook is running on colab
##  - (run_on_colab) = False : if notebook is running locally
'''
run_on_colab = True

'''
##  - (download_camem_bert_for_extractive_question_answering) = True : if Camem-BERT Model needs to get download from given checkpoint
##  - (download_camem_bert_for_extractive_question_answering) = False : if Camem-BERT Model is already downloaded and its path is step
'''
download_camem_bert_for_extractive_question_answering = True

'''
##  - (download_distil_bert_for_extractive_question_answering) = True : if Distil-BERT Model needs to get download from given checkpoint
##  - (download_distil_bert_for_extractive_question_answering) = False : if Distil-BERT Model is already downloaded and its path is step
'''
download_distil_bert_for_extractive_question_answering = True


Installing required dependencies and setuping base paths

In [2]:
if run_on_colab:
    from google.colab import drive
    from google.colab import files
    base_path = '/content/drive'
    drive.mount(base_path)
    base_path = base_path + '/My Drive/'

    ## UPLOADING REQUIREMENTS.TXT
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

    ## INSTALLING ALL DEPENDENCIES
    !pip3 install -r requirements_task2.txt

else:
    base_path = '/Users/fdhanani/Desktop/JOKER CLEF/'
    ## INSTALLING ALL DEPENDENCIES
    !pip3 install -r requirements_task2.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving requirements_task2.txt to requirements_task2 (1).txt
User uploaded file "requirements_task2.txt" with length 1627 bytes
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Collection of all the imports required to run this noteboook

In [3]:
import json
import torch
import collections
import numpy as np
import pandas as pd 
from tqdm import tqdm
from functools import partial
from torch.optim import AdamW
from sklearn.utils import shuffle
from accelerate import Accelerator
from torch.utils.data import DataLoader
from translate.storage.tmx import tmxfile
from sklearn.model_selection import train_test_split

from datasets import load_dataset, Features, Value, load_metric
from transformers import AutoTokenizer, default_data_collator, AutoModelForQuestionAnswering, get_scheduler, pipeline

Before executing the subsequent notebook cells, please clone all the related files from the [official Github repository](https://github.com/FarhanDhanani/joker-clef-22-FAST-MT). And ensure the mentioned directory structure is set up correctly either on the Google Drive if you are executing on the collab or your local system, in case you are running it locally. 

**DIRECTORY STRUCTURE:** 
```
.
├── JOKER_CLEF_TASK_1.ipynb
├── JOKER_CLEF_TASK_2.ipynb
├── JOKER_CLEF_TASK_3.ipynb
├── requirements_task1.txt
├── requirements_task2.txt
├── requirements_task3.txt
│── Readme.md
├── dataset
│   └── JOKER
│       ├── Task 1
│       │   ├── test
│       │   │   ├── check_format.py
│       │   │   ├── joker_task1_en_test.csv
│       │   │   ├── joker_task1_en_test.json
│       │   │   └── joker_task1_en_testfull.json
│       │   └── train
│       │       ├── FT_Model
│       │       │   ├── FINE_TUNED_MODELS_FOR_TASK1_WILL_SAVE_HERE
|       |       |   ├── TFDistilBertForSequenceClassification
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassification
│       │       │   ├── TFDistilBertForSequenceClassificationOfConventionalType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfConventionalType
│       │       │   ├── TFDistilBertForSequenceClassificationOfCulturalReferenceType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfCulturalReferenceType
│       │       │   ├── TFDistilBertForSequenceClassificationOfHorizontalVerticalType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfHorizontalVerticalType
│       │       │   ├── TFDistilBertForSequenceClassificationOfManipulationType
│       │       │   │   ├── finetuned model TFDistilBertForSequenceClassificationOfManipulationType
│       │       │   ├── bert-finetuned-ner-JOKER_task1
│       │       │   │   ├── finetuned model ner-JOKER_task1
│       │       │   ├── bert-finetuned-ner-location-JOKER_task1
│       │       │   │   ├── finetuned model location-JOKER_task1
│       │       ├── joker_task1_en_train.csv
│       │       ├── joker_task1_en_train.json
│       │       ├── joker_task1_en_trainfull.json
│       │       ├── train_val_split
│       │       │   ├── train1.csv
│       │       │   ├── train1.json
│       │       │   ├── val1.csv
│       │       │   └── val1.json
│       ├── Task 2
│       │   ├── test
│       │   │   ├── CAMEM-solutions_joker_task2_test_context_aware_data_where_answers_exist.json
│       │   │   ├── DISTIL-solutions_joker_task2_test_context_aware_data_where_answers_exist.json
│       │   │   ├── check_format.py
│       │   │   ├── joker_task2_test.csv
│       │   │   ├── joker_task2_test.json
│       │   │   ├── joker_task2_test_context_aware_data_where_answers_exist.csv
│       │   └── train
│       │       ├── FT_Model
│       │       │   ├── camembert-base-squadFR-fquad-piaf
│       │       │   │   ├── Fine-tuned camembert model will save here
│       │       │   └── distilbert-base-cased-distilled-squad
│       │       │       ├── Fine-tuned distilbert model will save here
│       │       ├── joker_task2_train.csv
│       │       ├── joker_task2_train.json
│       │       ├── joker_task2_train_context_aware_data_where_answers_exist.csv
│       │       └── train_val_split
│       │           ├── train2.csv
│       │           └── val2.csv
│       ├── Task 3
│       │   ├── test
│       │   │   ├── check_format.py
│       │   │   ├── joker_task3_test.csv
│       │   │   └── joker_task3_test.json
│       │   └── train
│       │       ├── Helsinki-NLP-performance.json
│       │       ├── joker_task3_train.csv
│       │       ├── joker_task3_train.json
│       │       ├── t5-base-performance.json
│       │       ├── t5-large-performance.json
│       │       └── t5-small-performance.json
│       ├── csv
│       │   ├── context
│       │   │   ├── c_ref.csv
│       │   │   ├── c_ref_test.csv
│       │   │   └── c_ref_train.csv
│       │   ├── en-fr_books.csv
│       │   ├── en-fr_bookshop.csv
│       │   ├── en-fr_opensub.csv
│       │   ├── en-fr_ted.csv
│       │   ├── en-fr_wiki.csv
│       │   ├── en-fr_wikipedia.csv
│       │   ├── en_processed
│       │   │   ├── en-fr_books.csv
│       │   │   ├── en-fr_bookshop.csv
│       │   │   ├── en-fr_opensub.csv
│       │   │   ├── en-fr_ted.csv
│       │   │   ├── en-fr_wiki.csv
│       │   │   └── en-fr_wikipedia.csv
│       │   └── extractVocab
│       │       ├── test_nonPok_en-fr_books.json
│       │       ├── test_nonPok_en-fr_bookshop.json
│       │       ├── test_nonPok_en-fr_opensub.json
│       │       ├── test_nonPok_en-fr_ted.json
│       │       ├── test_nonPok_en-fr_wiki.json
│       │       ├── test_nonPok_en-fr_wikipedia.json
│       │       ├── train_nonPok_en-fr_books.json
│       │       ├── train_nonPok_en-fr_bookshop.json
│       │       ├── train_nonPok_en-fr_opensub.json
│       │       ├── train_nonPok_en-fr_ted.json
│       │       ├── train_nonPok_en-fr_wiki.json
│       │       └── train_nonPok_en-fr_wikipedia.json
│       ├── en-fr.tmx_wiki.gz
│       ├── en-fr_books.tmx
│       ├── en-fr_books.tmx.gz
│       ├── en-fr_bookshop.tmx
│       ├── en-fr_bookshop.tmx_2.gz
│       ├── en-fr_opensub.tmx
│       ├── en-fr_opensub.tmx.gz
│       ├── en-fr_ted.tmx
│       ├── en-fr_ted.tmx.gz
│       ├── en-fr_wiki.tmx
│       ├── en-fr_wikipedia.tmx
│       ├── en-fr_wikipedia.tmx.gz
│       ├── dragons.csv
│       ├── task2_pokemon_en.json
│       └── task2_pokemon_fr.json
├── models
|   |── Downloaded Pre-trained Models will get save here
│   ├── Helsinki-NLP-opus-mt-en-fr
│   │   
│   ├── bert-base-cased
│   │   
│   ├── camembert-base-squadFR-fquad-piaf
│   │   
│   ├── distilbert-base-cased-distilled-squad
│   │   
│   ├── distilbert-base-uncased
│   │   
│   ├── mt5-small
│   │   
│   ├── roberta-base
│   │   
│   └── t5-base
│       
├── .
```

# **TASK 2: TRANSLATE SINGLE WORDS (NOUNS) CONTAINING WORDPLAY.**

Train data format: List of translated wordplay instances in a JSON format or a CSV file (for manual runs) with the following fields:

    id: a unique wordplay identifier
    en: wordplay text in English (source)
    fr: wordplay text in French (target)

Example:

    [{"id":"noun_1","en":"Ambipom","fr":"Capidextre"}]

Test data input format: List of wordplay instances to translate in a JSON format or a CSV file (for manual runs) with the following fields:

    id: a unique wordplay identifier
    en: wordplay text in English (source)

Input example:

    [{"id":"noun_1185","en":"Fungun"}]

Test data output format:

List of wordplay instances to be translated in a JSON format or a CSV file (for manual runs) with the following fields:

    RUN_ID: Run ID starting with team_id_ (as registered at the CLEF website)
    MANUAL: Whether the run is manual {0,1}
    id: a unique wordplay identifier
    en: wordplay text in English (source)
    fr: wordplay text in French (target)

Output example:List of wordplay instances to be translated in a JSON format or a CSV file (for manual runs) with the following fields:

    [{"RUN_ID":"OFFICIAL","MANUAL":1,"id":"noun_1","en":"Ambipom","fr":"Capidextre"}]


**Evaluation**. Human evaluators will manually annotate the submitted translations according to both subjective measures and according to more concrete features such as whether wordplay exists in the target text, whether it corresponds to the type used in the source text, whether the target text preserves the semantic field, etc.

**Result submission**. Participants should put their run results into the folder Documents created for their user and submit them by email to contact@joker-project.com. The email subject has to be in the format [CLEF TASK 2] TEAM_ID.


## LOADING THE DATASETS

LOADING THE TRAIN DATA-SET FOR TASK-2

In [4]:
'''
## The train data file were given in the form of json format.
## We have load them into pandas dataframe for further analysis.
'''

path = "/dataset/JOKER/Task 2/train/joker_task2_train.csv"
data_set_2 = pd.read_csv(base_path+path)
data_set_2.head(5)

,id,en,fr
0,noun_1,Ambipom,Capidextre
1,noun_2,Dartrix,Efflèche
2,noun_3,Malamar,Sepiatroce
3,noun_4,Bounsweet,Croquine
4,noun_5,Obelix,Obélix


THE STRUCTURE OF THE TRAIN DATA-SET

In [5]:
'''
 ## Determining the structure of the training data set
'''
data_set_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164 entries, 0 to 1163
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1158 non-null   object
 1   en      1164 non-null   object
 2   fr      1164 non-null   object
dtypes: object(3)
memory usage: 27.4+ KB


LOADING THE TEST DATA-SET FOR TASK-2

In [6]:
'''
## The test data file were given in the form of json format.
## We have load them into pandas dataframe for further analysis.
'''
path = "dataset/JOKER/Task 2/test/joker_task2_test.csv"
test_data_set_2 = pd.read_csv(base_path+path)
test_data_set_2.head(5)

,id,en
0,noun_1161,Orbeetle
1,noun_1162,Gossifleur
2,noun_1163,Eldegoss
3,noun_1164,Crabominable
4,noun_1165,Ribombee


In [7]:
'''
 ## Determining the structure of the testing data set.
'''
test_data_set_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      284 non-null    object
 1   en      284 non-null    object
dtypes: object(2)
memory usage: 4.6+ KB


## EDA FOR THE LOADED DATASET

TOTAL NUMBER OF RECORDS IN THE GiVEN TESTING DATA-SET

In [8]:
'''
 ## The number of total records in the given testing data set
'''
len(test_data_set_2)

284

NUMBER OF MISSING VALUES IN EACH COLUMN OF THE TEST DATA-SET

In [9]:
test_data_set_2.isnull().sum()

id    0
en    0
dtype: int64

TOTAL NUMBER OF RECORDS IN TRAIN DATASET

In [10]:
'''
 ## The number of total records in the given training data set
'''
len(data_set_2)

1164

NUMBER OF TOTAL MISSING VALUES IN EACH COLOUMN

In [11]:
'''
 ## Finding out total Number of missing cells in the given training data set
'''
data_set_2.isnull().sum()

id    6
en    0
fr    0
dtype: int64

EXPLORIND ID VARIABLE IN THE TRAINING DATA-SET

Usually ID should be non-null unique identifier for each record (row) in the data-set. Here, in the provided training data, we can observe that ID is unique but its null (absent) for six records.

 - TOTAL NUMBER OF RECORDS: 1164

 - DISTINCT NUMBER OF IDs: 1158

 - RECORDS WITH MISSING IDs: 6

In [12]:
'''
 ## Finding out the description data related to id variable
'''
data_set_2.id.describe()

count       1158
unique      1158
top       noun_1
freq           1
Name: id, dtype: object

EXPLORING EN COLOUMN

In [13]:
'''
 ## Finding out the description data related to en variable
'''
data_set_2.en.describe()

count         1164
unique        1152
top       official
freq             3
Name: en, dtype: object

NOT ALL VALUES IN THE EN COLUMN ARE UNIQUE

Usually, as this task is predicting the parallel translation in french for a given word in English. So, ideally there must not be any diplicates, and specifically the duplicates words in English should not have different parallel translation in French. Because the translation is based on single given word so a determinsitic model can never produce two different outputs for a samen given input. In other case, we can also assume that for a given English word there exist more than one possible translated word in French.

In [14]:
'''
 ## Finding duplicate English words in the given training data.
'''
dups = data_set_2[data_set_2.duplicated(subset=['en'], keep=False)]
dups.sort_values(['en'], ascending=[1])

,id,en,fr
60,noun_61,Crustacius,Gracchus Cétinconcensus
676,noun_674,Crustacius,Gracchus Cétinconsensus
1157,noun_1155,Elimentaler,Fromager
1065,noun_1063,Elimentaler,Demmentaleur
819,noun_817,Face Melter,Amplificatueur
1114,noun_1112,Face Melter,Défiguratrice
656,noun_654,Fightsabre,Épée-flingue
650,noun_648,Fightsabre,Mitrailleuse Jedi
169,noun_170,Glummy,Savourable
253,noun_252,Glummy,Miamifique


FINDING RECORDS IN WHICH EN COLUMN HAS NUMERIC VALUES

There are zero number of records in which English coloumn contains a numeric value.

In [15]:
'''
 ## Sanity check to verify if there exist any invalid numeric entry in the
 ## English column of the given training data-set
'''
data_set_2[pd.to_numeric(data_set_2['en'], errors='coerce').notnull()]

,id,en,fr


EXPLORING FR COLOUMN

In [16]:
'''
 ## Finding out the description data related to fr variable
'''
data_set_2.fr.describe()

count     1164
unique    1163
top          2
freq         2
Name: fr, dtype: object

NOT ALL VALUES IN THE FR COLUMN ARE UNIQUE

The following provided French translations for the English word are not valid they must get filtered from the data.

In [17]:
'''
 ## Finding duplicate French words in the given training data.
'''
dups = data_set_2[data_set_2.duplicated(subset=['fr'], keep=False)]
dups.sort_values(['fr'], ascending=[1])

,id,en,fr
179,NaN,official,2
321,NaN,official,2


FINDING RECORDS IN WHICH FR COLUMN HAS NUMERIC VALUES

There are three records in which parallel French translations against an English word is numeric. These are invalid records which either needs to get ccorrected or filtered out.

In [18]:
'''
 ## Sanity check to verify if there exist any invalid numeric entry in the
 ## French column of the given training data-set
'''
data_set_2[pd.to_numeric(data_set_2['fr'], errors='coerce').notnull()]

,id,en,fr
179,NaN,official,2
214,NaN,official,1
321,NaN,official,2


## PRE-PROCESSING AND BUILDING MODELS

After analyzing the data set, we can observe that it contains English nouns from movies, cartoons, and animes, along with their corresponding translations in French versions. For example, we can look at the first record from the data set which is **"Ambipom."** It is the English name of a pokemon, and the following pictures show its appearance.

<center><img src="https://img.pokemondb.net/artwork/large/ambipom.jpg"/></center>

So usually, the anime and movie writers choose the names of their movie characters very carefully to advertise the idea of the role their characters will play in the movie. So, in this case, **"Ambi"** is a prefix from Latin that means **both**. The other sub-word **"pom"** refers to the word **"pom-pom"** (the stuff used by cheerleaders for decorating their hands and emphasizing their hand movements). Collectively, the word **"Ambipom"** reflects the idea of someone who wears a pom-pom in both hands all the time and has a very cheerful feminine nature. The challenge given in the task is to capture this pun in given English nouns taken from popular movies, anime, and cartoon shows and learn to output their corresponding translations in French.

In [19]:
data_set_2.iloc[0]

id        noun_1
en       Ambipom
fr    Capidextre
Name: 0, dtype: object

We have mapped the task of learning the mapping between English nouns and their corresponding French Translations into the extractive Question/Answer problem. We have transformed the two-column-based tabular data set for task2 into a classical styled extractive Q/A problem data set by using the English/French parallel sentence pairs (subtitles, books, and wiki) from the [OPUS open-source parallel corpus](https://opus.nlpl.eu/). In such a way that, for each English/French noun pair mentioned in the Joker CLEF task2 data set, we have collected English/French pairs text from the downloaded English/French OPUS parallel corpus, which contains those nouns in them. The deep learning models have to use each English noun from the Joker CLEF task2 data set as a query and the corresponding French text as the context, which somewhere holds the translation of the queried English noun. And now, the task for the deep learning models is to locate the exact position of the French translation in the French text for the queried English noun. 


So as an example, we can consider the following fourth record where we have to learn the translation of the English noun **"Obelix"** to French version **"Obélix."**

| id | en      | fr |
| ----------- | ----------- | ----------- |
| 4  | Obelix      | Obélix |

For learning this translation, we will utillize the developed Extractive Q/A styled data set, which will have following entry for the given English/French nouns pair.

| id | context | question | answers |
| :---   | :---:     |    :----:   | :---:  |
| 17  | astérix et obélix ne devraient plus quitter le village. | Obelix      | {"text": ["ob\u00e9lix"], "answer_start": [11]} |

In this way, the task of deep learning models is to learn to output the values of the answers column after processing the given context and question in its input.



For more details about how we have converted all the English/French noun pairs provided in the training data-set of Joker CLEF task2 in the Extractive Q/A style data set, please check the notebook section [Conversion of two-column based tabular data set into Extractive Q/A style data set](#joker-clef-task2-processing) 

In [20]:
'''
 ## Exploring English/French noun pair given in the training data set.
'''
data_set_2.iloc[4]

id    noun_5
en    Obelix
fr    Obélix
Name: 4, dtype: object

In [21]:
'''
 ## Exploring the saved Q/A style contextual reference for each English noun in the training data set.
'''
col_list = ['id', 'context', 'question', 'answers']
train_dataset_path = "/dataset/JOKER/Task 2/train/joker_task2_train_context_aware_data_where_answers_exist.csv"
data_set_with_context_ref = pd.read_csv(base_path+train_dataset_path, usecols=col_list)
data_set_with_context_ref.iloc[17]

id                                                         17
context     astérix et obélix ne devraient plus quitter le...
question                                               obelix
answers       {"text": ["ob\u00e9lix"], "answer_start": [11]}
Name: 17, dtype: object

### PRE-PROCESSING THE PREPARED TRAINING DATA-SET TO LEARN TO LOCATE THE FRENCH TRANSLATION FOR THE GIVEN ENGLISH NOUN IN THE FORM OF QUERY IN THE FRENCH TEXT CONTEXT

First, we will split the train data set into training, testing, and validation sets to evaluate and track our model's performance. The joker CLEF task2 team has provided us with the test data set, but we don't know the real ground truth answers for them. We actually need to submit the predictions for the English nouns given in the task-2 test data set. So, the purpose of making an additional test set here is just to estimate the model performance on the unseen records and compare their performance with other models. 


Plus, we have created this test set by keeping some data aside from the actual training set. Therefore, we also have the ground-truth answers for this synthetic test set. And now, we can use this test set to evaluate the model's performance on the unseen instances and also to compare different models' performances.

In [22]:
'''
 ## Training and Validation Split
'''
col_list = ['id', 'context', 'question', 'answers']
train_dataset_path = "/dataset/JOKER/Task 2/train/joker_task2_train_context_aware_data_where_answers_exist.csv"
data_set_with_context_ref = pd.read_csv(base_path+train_dataset_path, usecols=col_list)
data_set_with_context_ref = shuffle(data_set_with_context_ref)
train_2, test_2 = train_test_split(data_set_with_context_ref, test_size=0.1, shuffle=True)
train_2, val_2 = train_test_split(train_2, test_size=0.2, shuffle=True)

'''
 ## Saving the split into the file system.
'''
save_split_path = "/dataset/JOKER/Task 2/train/train_val_split/"
train_2.to_csv(base_path+save_split_path+"train2.csv",columns =col_list)
val_2.to_csv(base_path+save_split_path+"val2.csv", columns=col_list)
print(len(train_2), len(val_2))

1563 391


Next, we will convert the developed train and validation CSVs into Dataset, which later gets utilized by the deep-learning models.

In [23]:
## FEATURES FOR DATA-SET
ft = Features({
    'id': Value(dtype='int64', id=None),
    'context': Value(dtype='string', id=None),
    'question': Value(dtype='string', id=None),
    'answers': Value(dtype='string', id=None),
})

## DATA FILES FOR DATA-SET
data_files = {
    "train": base_path+save_split_path+"train2.csv", 
    "validation": base_path+save_split_path+"val2.csv"
}

## LOADING THE DATA FILES INTO DATA SET
dataset_2 = load_dataset('csv', data_files=data_files, features=ft)
dataset_2

Using custom data configuration default-25ec87105edd7413


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-25ec87105edd7413/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1563
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 391
    })
})

Let's have a quick glance at the loaded data set and its contents.

In [24]:
'''
 ## Exploring 6th index of the loaded data set
'''
index = 3
print("Context: ", dataset_2["train"][index]["context"])
print("Question: ", dataset_2["train"][index]["question"])
print("Answer: ", dataset_2["train"][index]["answers"])

Context:  j'en fais le vœu solennel, j'irai au pays des vikings et je ramènerai goudurix avant la prochaine pleine lune.
Question:  justforkix
Answer:  {"text": ["goudurix"], "answer_start": [70]}


While training the deep learning models for extractive question answering, it is necessary to ensure that there must be only one possible answer for each of the given records. We have assured this while transforming the [two-column-based tabular data set provided by the Joker CLEF task2 team into an extractive Q/A styled data set](#joker-clef-task2-processing). But here, we can also re-verify this again with the following code snippet.

In [25]:
'''
 ## Verifying that no records in the data set have more than one possible answer or no available answer at all
'''
dataset_2["train"].filter(lambda x: len(json.loads(x["answers"])["text"]) != 1)

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 0
})

Next, we will tokenize the entire training data set so that our deep learning model can consume it.

In [26]:
'''
 ## Defining the max length and stride length for tokenizing the data set.
'''
max_length = 256
stride = 64


'''
 ## The function takes an example training record as its input and returns its aligned tokenized form as its output.
 
 ## INPUT:
 ## Example record from the training data set
 
 ## OUTPUT:
 ## Aligned tokenized form of the input training record
'''
def preprocess_training_examples(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = json.loads(answers[sample_idx])
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

We will define a similar kind of function for tokenizing and aligning the validation data, just as we have defined for the training data.

In [27]:
'''
 ## The function takes an example validation record as its input and returns its aligned tokenized form as its output.
 
 ## INPUT:
 ## Example record from the validation data set
 
 ## OUTPUT:
 ## Aligned tokenized form of the input validation record
'''
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

Next, we will design a function that will be able to evaluate the model's prediction based on a given metric.

In [28]:
'''
 ## The function evaluates and scores the produced predictions emitted by the 
 ## deep learning model according to a given metric.
 
 ## INPUTS:
 ## START LOGITS: defining predicted starting point for the answer in the provided context
 ## END LOGITS: defining predicted ending point for the answer in the provided context
 ## FEATURES: tokenized data set developed earlier from which we need to extract basic features
 ## EXAMPLES: individual examples in the data set which needs to get evaluated
 ## N_BEST: N number of best predictions which needs to be consider
 ## MAX_ANSWER_LENGTH: Number of expected max length for the produceed answer
 
 ## OUTPUTS:
 ## EVALUATED SOCORES for the generated prediction by model for the given input and context
'''
def compute_metrics(start_logits, end_logits, features, examples, metric, n_best = 1, max_answer_length = 20):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)
      
    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if not offsets[start_index] or not offsets[end_index]:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": json.loads(ex["answers"])} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers), predicted_answers, theoretical_answers

### PREPARING [CAMEMBERT](https://huggingface.co/etalab-ia/camembert-base-squadFR-fquad-piaf) TO GET FINETUNED ON THE DEVELOPED DATA SET FOR JOKER CLEF TASK-2

We have selected the [CamemBERT model](https://huggingface.co/etalab-ia/camembert-base-squadFR-fquad-piaf) from the hugging face repository to get fine-tuned on the developed data set for the Joker CLEF task 2 in extractive Q/A style. Later we will use the fine-tuned version of this model to generate the translations for the English nouns provided in the test set of the Joker CLEF task 2 data set.

In [29]:
'''
 ## Loading the tokenizer for CamemBERT
'''
model_checkpoint = "etalab-ia/camembert-base-squadFR-fquad-piaf" if download_camem_bert_for_extractive_question_answering else "models/camembert-base-squadFR-fquad-piaf"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


'''
 ## Exploring the working of loaded tokenizer
'''
context = dataset_2["train"][6]["context"]
question = dataset_2["train"][6]["question"]

inputs = tokenizer(question, context, max_length=384, 
                   truncation="only_second", stride=128, 
                   return_overflowing_tokens=True, return_offsets_mapping=True,)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))
    
inputs.keys()

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

<s> electric</s></s> gec alsthom est une entreprise commune néer­landaise créée par general electric company et alcatel alsthom cge.</s>


dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

Now, we will process each input record of the training and test data set with the previously defined preprocess_training_examples and preprocess_validation_examples function to generate the aligned tokenized dataset. 

In [30]:
'''
 ## Mapping each record of the training data set on the preprocess_training_examples function.
'''
train_dataset = dataset_2["train"].map(
    partial(preprocess_training_examples, tokenizer=tokenizer),
    batched=True,
    remove_columns=dataset_2["train"].column_names,)

len(dataset_2["train"]), len(train_dataset)

  0%|          | 0/2 [00:00<?, ?ba/s]

(1563, 1846)

In [31]:
'''
 ## Mapping each record of the validation data set on the preprocess_validation_examples function.
'''
validation_dataset = dataset_2["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset_2["validation"].column_names,)
len(dataset_2["validation"]), len(validation_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

(391, 443)

We will define data loaders for our prepared training and validation data sets. Plus, we will also enable torch format on the training and validation data set.

In [32]:
'''
 ## Set torch format on training and validation data set. 
 ## Plus, removing un-used columns names from the validation data set.
'''
train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

'''
 ## Initializing data loaders for the developed train and validation data set. 
'''
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

Almost all things are ready till here. Now, we can load the [CamemBERT](https://huggingface.co/etalab-ia/camembert-base-squadFR-fquad-piaf) model and the Adam optimizer.

In [33]:
'''
 ## Loading the CamemBERT model and Adam optimizer
'''
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
optimizer = AdamW(model.parameters(), lr=2e-5)

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

We will be using the accelerator API to fine-tune our model on the developed data set. So, let's create an object for the accelerator API, and initialize the schedulers with other hyper-parameters to trigger the training.

In [34]:
'''
 ## Preparing the accelerator API for starting the training.
'''
accelerator = Accelerator(fp16=True)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

In [35]:
'''
 ## Initializing hyper-parameters and the schedulers
'''
num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,)

Finally, we will define the training loop to fine-tune the loaded model

In [36]:
output_dir = base_path+"/dataset/JOKER/Task 2/train/FT_Model/camembert-base-squadFR-fquad-piaf"

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics, pridictions, validations = compute_metrics(
        start_logits, end_logits, validation_dataset, dataset_2["validation"],
        load_metric("squad"))
    
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

100%|██████████| 231/231 [00:38<00:00,  6.35it/s]

Evaluation!



100%|██████████| 56/56 [00:02<00:00, 23.06it/s]



100%|██████████| 391/391 [00:00<00:00, 1627.32it/s]


epoch 0: {'exact_match': 93.60613810741688, 'f1': 94.33077578857629}


### PREPARING [DistilBERT](https://huggingface.co/distilbert-base-cased-distilled-squad) TO FINETUNE MODEL ON THE DEVELOPED DATA SET FOR JOKER CLEF TASK-2

We have selected the [DistilBERT model](https://huggingface.co/distilbert-base-cased-distilled-squad) from the hugging face repository to get fine-tuned on the developed data set for the Joker CLEF task 2 in extractive Q/A style. Later we will use the fine-tuned version of this model to generate the translations for the English nouns provided in the test set of the Joker CLEF task 2 data set.

In [37]:
'''
 ## Loading the tokenizer for DistilBERT
'''
model_checkpoint = "distilbert-base-cased-distilled-squad" if download_distil_bert_for_extractive_question_answering else "models/distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


'''
 ## Exploring the working of loaded tokenizer
'''
context = dataset_2["train"][6]["context"]
question = dataset_2["train"][6]["question"]

inputs = tokenizer(question, context, max_length=384, 
                   truncation="only_second", stride=128, 
                   return_overflowing_tokens=True, return_offsets_mapping=True,)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))
    
inputs.keys()

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

[CLS] electric [SEP] gec alsthom est une entreprise commune néerlandaise créée par general electric company et alcatel alsthom cge. [SEP]


dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

Now, we will process each input record of the training and test data set with the previously defined preprocess_training_examples and preprocess_validation_examples function to generate the aligned tokenized dataset.

In [38]:
'''
 ## Mapping each record of the training data set on the preprocess_training_examples function.
'''
train_dataset = dataset_2["train"].map(
    partial(preprocess_training_examples, tokenizer=tokenizer),
    batched=True,
    remove_columns=dataset_2["train"].column_names,)

len(dataset_2["train"]), len(train_dataset)

  0%|          | 0/2 [00:00<?, ?ba/s]

(1563, 1827)

In [39]:
'''
 ## Mapping each record of the validation data set on the preprocess_validation_examples function.
'''
validation_dataset = dataset_2["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset_2["validation"].column_names,)
len(dataset_2["validation"]), len(validation_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

(391, 436)

We will define data loaders for our prepared training and validation data sets. Plus, we will also enable torch format on the training and validation data set.

In [40]:
'''
 ## Set torch format on training and validation data set. 
 ## Plus, removing un-used columns names from the validation data set.
'''
train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

'''
 ## Initializing data loaders for the developed train and validation data set. 
'''
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

Almost all things are ready till here. Now, we can load the [DistilBERT](https://huggingface.co/distilbert-base-cased-distilled-squad) model and the Adam optimizer.

In [41]:
'''
 ## Loading the DistilERT model and Adam optimizer
'''
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
optimizer = AdamW(model.parameters(), lr=2e-5)

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

We will be using the accelerator API to fine-tune our model on the developed data set. So, let's create an object for the accelerator API, and initialize the schedulers with other hyper-parameters to trigger the training.

In [42]:
'''
 ## Preparing the accelerator API for starting the training.
'''
accelerator = Accelerator(fp16=True)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

In [43]:
'''
 ## Initializing hyper-parameters and the schedulers
'''
num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,)

Finally, we will define the training loop to fine-tune the loaded model

In [44]:
output_dir = base_path+"/dataset/JOKER/Task 2/train/FT_Model/distilbert-base-cased-distilled-squad"

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics, pridictions, validations = compute_metrics(
        start_logits, end_logits, validation_dataset, dataset_2["validation"],
        load_metric("squad"))
    
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)


100%|██████████| 231/231 [01:31<00:00,  2.52it/s]

100%|█████████▉| 228/229 [00:20<00:00, 10.93it/s]

Evaluation!


100%|██████████| 391/391 [00:00<00:00, 1690.68it/s]


epoch 0: {'exact_match': 97.9539641943734, 'f1': 98.05626598465473}


### COMPARING PERFORMANCE OF FINE-TUNED MODELS 

In [45]:
'''
 ## Loading the fine-tuned CamemBRET model
'''
output_dir = base_path+"/dataset/JOKER/Task 2/train/FT_Model/camembert-base-squadFR-fquad-piaf"
FT_model_camemBERT = pipeline(task= "question-answering", model= output_dir, tokenizer = output_dir)

'''
 ## Loading the fine-tuned DistilBRET model
'''
output_dir = base_path+"/dataset/JOKER/Task 2/train/FT_Model/distilbert-base-cased-distilled-squad"
FT_model_distilBERT = pipeline(task= "question-answering", model= output_dir, tokenizer = output_dir)

perfect_matches = 0
camemBertPredictions = []
camemBertActualLabels = []

for index, row in tqdm(test_2.iterrows()):
    context = row.context.lower().strip()
    question = row.question.lower().strip()
    prediction = FT_model_camemBERT(question=question, context=context)
    
    camemBertPredictions.append(prediction["answer"])
    camemBertActualLabels.append(json.loads(row.answers)['text'][0])
    
    if (prediction["answer"] == json.loads(row.answers)['text'][0]):
        perfect_matches +=1
print("CamemBERT PERFECT MATCHES: ", perfect_matches, "OUT OF TOTAL RECORDS: ", len(test_2))


perfect_matches = 0
distilBertPredictions = []
distilBertActualLabels = []
for index, row in tqdm(test_2.iterrows()):
    context = row.context.lower().strip()
    question = row.question.lower().strip()
    prediction = FT_model_distilBERT(question=question, context=context)
    
    distilBertPredictions.append(prediction["answer"])
    distilBertActualLabels.append(json.loads(row.answers)['text'][0])
    
    if (prediction["answer"] == json.loads(row.answers)['text'][0]):
        perfect_matches +=1

print("DistilBERT PERFECT MATCHES: ", perfect_matches, "OUT OF TOTAL RECORDS: ", len(test_2))

8it [00:00,  7.99it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)

218it [01:12,  3.01it/s]


CamemBERT PERFECT MATCHES:  96 OUT OF TOTAL RECORDS:  218


218it [00:38,  5.66it/s]

DistilBERT PERFECT MATCHES:  209 OUT OF TOTAL RECORDS:  218


### EXECUTING FINE-TUNED MODELS TO GENERATE TRANSLATIONS FOR ENGLISH NOUNS FROM THE TEST DATA-SET

Lastly, we have applied the same approach to transform the English nouns in the test data set provided by the Joker CLEF task2 team into extractive Q/A styled data sets by using the OPUS English/French parallel corpus. The only difference is that this time we will not the answers column. So, during this conversion, we have only ensured the existence of English nouns in the English version text of the English/French parallel corpus to filter out the relevant pairs. We have assumed that the French text in the extracted pair contains the relevant translation for the English noun that exists in the English text.

So as an example, we can consider the following record from test data set where we have to predict the french translation of the English noun **"Loompaland."**

| id | en      | 
| ----------- | ----------- | 
| 18  | Loompaland      | 

For predicting this translation, we will utillize the developed Extractive Q/A styled data set, which will have following entry for the mentioned English noun.

| id | context | question |  
| :---   | :---:     |    :----:   | 
| 17  | j'étais venu à lumpaland pour chercher de nouvelles saveurs. | loompaland |

In this way, the task of fine-tuned deep learning model is now to output the location of the answer in the given context. And if we consider the above example, then the fine-tuned model has to spot the word **lumpaland** as an answer.

For more details about how we have converted all the English nouns provided in the test data-set of Joker CLEF task2 in the Extractive Q/A style data set, please check the notebook section [Conversion of two-column based tabular data set into Extractive Q/A style data set](#joker-clef-task2-processing) 

In [46]:
'''
 ## Exploring the mentioned record from the example above in the test data set for Joker CLEF task2
'''
test_data_set_2.iloc[18]

id     noun_1179
en    Loompaland
Name: 18, dtype: object

In [47]:
'''
 ## Exploring the correesponding Q/A stuled entry for mentioned record in the example above
'''
cols = ["id", "context", "question"]
test_data_2 = pd.read_csv(base_path+
                          "/dataset/JOKER/Task 2/test/joker_task2_test_context_aware_data_where_answers_exist.csv", 
                          usecols = cols)
test_data_2.iloc[18]

id                                                         18
context     j'étais venu à lumpaland pour chercher de nouv...
question                                           loompaland
Name: 18, dtype: object

Finally, we will use the fine-tuned model to generate the predictions for all the English nouns provided in the test data set and are successfully converted into Q/A styled dataset.

In [48]:
'''
 ## Loading the Q/A styled transformed data set for the English nouns provided in the test data set 
 ## of Joker CLEF task2
'''
cols = ["id", "context", "question"]
test_data_2 = pd.read_csv(base_path+
                          "/dataset/JOKER/Task 2/test/joker_task2_test_context_aware_data_where_answers_exist.csv", 
                          usecols = cols)

'''
 ## Loading the fine-tuned CamemBRET model
'''
output_dir = base_path+"/dataset/JOKER/Task 2/train/FT_Model/camembert-base-squadFR-fquad-piaf"
FT_model = pipeline(task= "question-answering", model= output_dir, tokenizer = output_dir)

'''
 ## Generating translations
'''
predictions=[]
for index, row in tqdm(test_data_2.iterrows()):
    context = row.context.lower().strip()
    question = row.question.lower().strip()
    prediction = FT_model(question=question, context=context)
    predictions.append({"MANUAL":0,
                       "id":row.id,
                       "en":row.question,
                       "fr":prediction["answer"]})

'''
 ## Saving the generated translation
'''    
with open(base_path+"/dataset/JOKER/Task 2/test/CAMEM-solutions_joker_task2_test_context_aware_data_where_answers_exist.json", "w") as outfile:
    outfile.write(json.dumps(predictions, indent = 4))

6176it [20:59,  4.90it/s]


In [49]:
'''
 ## Loading the Q/A styled transformed data set for the English nouns provided in the test data set 
 ## of Joker CLEF task2
'''
cols = ["id", "context", "question"]
test_data_2 = pd.read_csv(base_path+
                          "/dataset/JOKER/Task 2/test/joker_task2_test_context_aware_data_where_answers_exist.csv", 
                          usecols = cols)

'''
 ## Loading the fine-tuned DistilBRET model
'''
output_dir = base_path+"/dataset/JOKER/Task 2/train/FT_Model/distilbert-base-cased-distilled-squad"
FT_model = pipeline(task= "question-answering", model= output_dir, tokenizer = output_dir)

'''
 ## Generating translations
'''
predictions=[]
for index, row in tqdm(test_data_2.iterrows()):
    context = row.context.lower().strip()
    question = row.question.lower().strip()
    prediction = FT_model(question=question, context=context)
    predictions.append({"MANUAL":0,
                       "id":row.id,
                       "en":row.question,
                       "fr":prediction["answer"]})

'''
 ## Saving the generated translation
'''    
with open(base_path+"/dataset/JOKER/Task 2/test/DISTIL-solutions_joker_task2_test_context_aware_data_where_answers_exist.json", "w") as outfile:
    outfile.write(json.dumps(predictions, indent = 4))

6176it [15:13,  6.76it/s]


In [56]:
path = "/dataset/JOKER/Task 2/test/DISTIL-solutions_joker_task2_test_context_aware_data_where_answers_exist.json"
file = open(base_path+path)
solutions = json.load(file)

path = "dataset/JOKER/Task 2/test/joker_task2_test.csv"
test_data_set_2 = pd.read_csv(base_path+path)

task_subimission_2 = []
run_id = "non-disclosable-numeric-UUID"
is_manual = 0

for index, row in tqdm(test_data_set_2.iterrows()):
    prediction = "NOT FOUND"
    for sol in solutions:
      if(row.en == sol['en']):
        prediction = sol['fr']
        break

    predictions_dict = {}
    predictions_dict["RUN_ID"] = run_id
    predictions_dict["MANUAL"] = is_manual
    predictions_dict["id"] = row.id
    predictions_dict["en"] = row.en
    predictions_dict["fr"] = prediction
    task_subimission_2.append(predictions_dict)

with open(base_path + "dataset/JOKER/Task 2/test/[CLEF TASK <2>] "+run_id+".json", 'w') as fp:
    json_object = json.dumps(task_subimission_2, indent = 4) 
    json.dump(json_object, fp)

284it [00:12, 22.78it/s]


## Transforming two-column-based tabular data set for English/French noun pairs into Extractive Q/A styled data set

We aim to map the JOKER CLEF task-2 to the extractive Question/Answer approach by using the English/French parallel sentence pairs (subtitles, books, and wiki) from the [OPUS open-source parallel corpus](https://opus.nlpl.eu/) where each pair must contain at least one English noun in the English version and the corresponding translation of the English noun in the French version. After that the processing of training is simple, we can treat the English noun as a question and the French text which contains its corresponding translation as its context. Now, the task is to extract the corresponding translation of the given English noun in the French text. Below, we have given the script to generate a simple CSV file from the TMX version of the English/French parallel corpus developed from ted talks downloaded from the [OPUS open-source parallel corpus](https://opus.nlpl.eu/). 

<a id='joker-clef-task2-processing'></a>

We can repeat this process for all the English/French parallel corpus available at the OPUS. Here in this notebook, we have only used the following listed English/French parallel corpus.

 - BOOKS
 - OPEN-SUB 
 - WIKIPEDIA
 - WIKIMDEIA
 - TED
 
 

In [ ]:
'''
 ## Conversion of English/French ted talks parllel corpus from tmx to csv
'''

## Opening tmx file
d_name = "en-fr_ted"
store_tmx_name = d_name+".tmx"
store_csv_name = d_name+".csv"
store_json_name = d_name+".json"
path = "dataset/JOKER/" + store_tmx_name
file_name = base_path + path
with open(file_name, 'rb') as fin:
    tmx_file = tmxfile(fin, 'en', 'fr')

## converting and saving it into csv file
en = []
fr = []
count = 0
for node in tqdm(tmx_file.unit_iter()):
    en.append(node.source)
    fr.append(node.target)

df = pd.DataFrame(list(zip([],[])), columns = ["en", "fr"])    
df['en'] = en
df['fr'] = fr
df.to_csv(base_path + "dataset/JOKER/csv/" + store_csv_name)


0it [00:00, ?it/s]
4648it [00:00, 46471.18it/s]
9527it [00:00, 47832.23it/s]
14373it [00:00, 48117.94it/s]
19264it [00:00, 48427.35it/s]
24107it [00:00, 47212.16it/s]
28959it [00:00, 47646.04it/s]
33728it [00:00, 46379.51it/s]
38375it [00:00, 46257.44it/s]
43324it [00:00, 47240.91it/s]
48274it [00:01, 47927.40it/s]
53072it [00:01, 47861.10it/s]
57862it [00:01, 47791.13it/s]
62670it [00:01, 47872.83it/s]
67459it [00:01, 46323.44it/s]
72103it [00:01, 46179.70it/s]
76823it [00:01, 46477.68it/s]
81725it [00:01, 47229.45it/s]
86576it [00:01, 47609.03it/s]
91488it [00:01, 48048.87it/s]
96297it [00:02, 44106.81it/s]
100772it [00:02, 43830.99it/s]
105878it [00:02, 45886.04it/s]
111009it [00:02, 47450.72it/s]
115795it [00:02, 47568.38it/s]
120776it [00:02, 48227.11it/s]
125618it [00:02, 47924.90it/s]
130539it [00:02, 48302.80it/s]
135495it [00:02, 48674.11it/s]
140507it [00:02, 49104.08it/s]
145440it [00:03, 49169.75it/s]
150361it [00:03, 47515.32it/s]
155127it [00:03, 45434.27it/s]
159696it [

It's important to note that the complete English/French parallel corpus is not useful for us. Because we only want the French version text for the given English nouns in the Joker-CLEF task-2 train/test data set for developing the contexts. So, the problem can be transformed into the extractive question answering where English nouns will serve as a query, and the task is to extract the translation of the provided English noun from the French version of the text. 

To implement such an approach, we have to extract a subset of the parallel corpus where the English version text must contain at least one English noun from the Joker-CLEF task-2 data set. Here we will assume that its corresponding French version will always have the translated version of the English noun in it for the Joker-CLEF task 2 test data set. So, to develop such sort of sub-set of English/French parallel corpus, we will iterate through all English nouns given in the training data-set by the JOKER-CLEF task-2 team and search them word by word in the English version text of the English/French parallel corpus. After that, we will create another data frame to store such pairs and use them further in our training.

To make the process of searching case-insensitive, we must first lower the English version text in the English/French parallel corpus and save it in a new CSV file on the disk.

In [ ]:
'''
 ## Converting text present in the English Column into lower case and saving the result in a new processed CSV file
'''

listed = df["en"].tolist()
pr_en = [ n.lower() for n in listed]
faster_search_df = pd.DataFrame(list(zip([],[])), columns = ["en", "fr"])
faster_search_df['en'] = pr_en
faster_search_df['fr'] = df["fr"].tolist()
faster_search_df.to_csv(base_path + "dataset/JOKER/csv/en_processed/" + store_csv_name)

In the EDA, we have also noticed that both the train and test data set of the JOKER CLEF task 2 consist of a lot of English nouns from the [Pokemon](https://pokemondb.net/pokedex/all) and [How to train your dragon series](https://howtotrainyourdragon.fandom.com/wiki/A_Hero%27s_Guide_to_Deadly_Dragons). Below, we have downloaded a list of pokemon names and the nouns from the "How to train your dragon series" to check the count of records collected from this list in the train/test data set provided by the JOKER CLEF task 2 team.

 FINDING FROM POKEMON DATASET HOW MANY RECORDS WE CAN USE FOR TRAINING FROM TRAINING DATASET

In [ ]:
'''
 ## Counting the number of records from the training dataset that are Pokemon names.
'''

## loading the pokemon names json file
path = "/dataset/JOKER/task2_pokemon_en.json"
file = open(base_path+path)
pokemonVocab = json.load(file)

## counting the number of English nouns present in the loaded json file of Pokemon names
count =0;
for index, row in data_set_2.iterrows():
    if (row.en in pokemonVocab):
        count = count+1
        pokemonVocab.index(row.en)
        
print("Count Of Pokemon Records In Train=",count, ", Total Records=", len(data_set_2.en), ", Percent=", count/len(data_set_2.en)*100)

Count Of Pokemon Records In Train= 520 , Total Records= 1164 , Percent= 44.67353951890035


FINDING FROM POKEMON DATASET FOR HOW MANY TESTING RECORDS WE CAN GENERATE TRANSLATIONS

In [ ]:
'''
 ## Counting the number of records from the test dataset that are Pokemon names.
'''

## loading the test data set
path = "dataset/JOKER/Task 2/test/joker_task2_test.csv"
test_data_set_2 = pd.read_csv(base_path+path)

## counting the number of English nouns from the test data set present in the loaded json file of Pokemon names
count =0;
for index, row in test_data_set_2.iterrows():
    if (row.en in pokemonVocab):
        count = count+1
        pokemonVocab.index(row.en)
        
print("Count Of Pokemon Records In Test=",count, ", Total Records=", len(test_data_set_2.en), ", Percent=", count/len(test_data_set_2.en)*100)

Count Of Pokemon Records In Test= 140 , Total Records= 284 , Percent= 49.29577464788733


FINDING FROM DRAGON DATASET HOW MANY RECORDS WE CAN USE FOR TRAINING FROM TRAINING DATASET

In [ ]:
'''
 ## Counting the number of English nouns from the training dataset that belongs to the "How to train your dragon" series.
'''

## loading the CSV file containing nouns from the "How to train your dragon" series. 
path = "dataset/JOKER/dragons.csv"
dragon_data_set = pd.read_csv(base_path+path, names=["en", "fr"])


## counting the number of English nouns from the train data set present in the loaded CSV file
dragon_dictionary_train = {}
listed = np.array([ n.lower() for n in dragon_data_set.en])
f = np.frompyfunc(lambda x,w: w in x, 2,1)
for index, row in tqdm(data_set_2.iterrows()):
    word = row.en.lower()
    finds = f(listed, word)
    dragon_dictionary_train[word] = finds.sum()

count=0
for key in dragon_dictionary_train:
    if(dragon_dictionary_train[key]>0):
        count = count+1
        
print("Count Of Dragon Records In Train=",count, ", Total Records=", len(data_set_2.en), ", Percent=", count/len(data_set_2.en)*100)



0it [00:00, ?it/s]
176it [00:00, 1754.79it/s]
369it [00:00, 1857.20it/s]
584it [00:00, 1989.46it/s]
787it [00:00, 2004.06it/s]
1164it [00:00, 1987.98it/s][A

Count Of Dragon Records In Train= 161 , Total Records= 1164 , Percent= 13.831615120274915


FINDING FROM DRAGON DATASET FOR HOW MANY TESTING RECORDS WE CAN GENERATE TRANSLATIONS

In [ ]:
'''
 ## Counting the number of English nouns from the test dataset that belongs to the "How to train your dragon" series.
'''

## loading the CSV file containing nouns from the "How to train your dragon" series. 
path = "dataset/JOKER/dragons.csv"
dragon_data_set = pd.read_csv(base_path+path, names=["en", "fr"])

## counting the number of English nouns from the test data set present in the loaded CSV file
dragon_dictionary_test = {}
listed = np.array([ n.lower() for n in dragon_data_set.en])
f = np.frompyfunc(lambda x,w: w in x, 2,1)
for index, row in tqdm(test_data_set_2.iterrows()):
    word = row.en.lower()
    finds = f(listed, word)
    dragon_dictionary_test[word] = finds.sum()

count=0
for key in dragon_dictionary_test:
    if(dragon_dictionary_test[key]>0):
        count = count+1
        
print("Count Of Dragon Records In Test=",count, ", Total Records=", len(test_data_set_2.en), ", Percent=", count/len(test_data_set_2.en)*100)


0it [00:00, ?it/s]
284it [00:00, 1865.66it/s]

Count Of Dragon Records In Test= 32 , Total Records= 284 , Percent= 11.267605633802818


Here we can conclude that almost 57% of the English given provided in the training data set for the Joker CLEF task2 belongs to the Pokemon and how to train your dragon series. Along with that almost 60% of the English nouns mentioned in the test data set belong to the same Pokemon and How to train your dragon series. 

So, it's important to consider that we must have to search the context for those English nouns in the downloaded English/French parallel corpus from the OPUS data set which does not belong to the Pokemon and How to train your dragon series.

In [ ]:
'''
 ## Calculation of the percentage of records from the training data set that do not belong
 ## to either the pokemon names or the nouns from the How to train your dragon series.
'''

filter_unresolved_train_records = []
for index, row in data_set_2.iterrows():
    if not ((row.en in pokemonVocab) or (dragon_dictionary_train[row.en.lower()] > 0)):
        filter_unresolved_train_records.append(row.en)
        
print("Count Of Records In Training that does not belong to either Pokemon or How to train your dragon series=",len(filter_unresolved_train_records), ", Total Records=", len(data_set_2.en),", Percent=", len(filter_unresolved_train_records)/len(data_set_2.en)*100)


Count Of Records In Training that does not belong to either Pokemon or How to train your dragon series= 483 , Total Records= 1164 , Percent= 41.49484536082475


In [ ]:
'''
 ## Calculation of the percentage of records from the test data set that do not belong
 ## to either the pokemon names or the nouns from the How to train your dragon series.
'''

filter_unresolved_test_records = []
for index, row in test_data_set_2.iterrows():
    if not ((row.en in pokemonVocab) or (dragon_dictionary_test[row.en.lower()] > 0)):
        filter_unresolved_test_records.append(row.en)

print("Count Of Records In Testing that does not belong to either Pokemon or How to train your dragon series=",len(filter_unresolved_test_records), ", Total Records=", len(test_data_set_2.en),", Percent=", len(filter_unresolved_test_records)/len(test_data_set_2.en)*100)


Count Of Records In Testing that does not belong to either Pokemon or How to train your dragon series= 113 , Total Records= 284 , Percent= 39.7887323943662


This analysis shows that we need to find almost 42% of English nouns from the training data set of the Joker CLEF task2. Plus, 40% of English nouns from the test data set provided by the Joker CLEF task2 team. 

Below, we have given a visual explanation of this concept as a sort of verification that our calculations are correct.

VERIFICATION FOR MATH IS CORRECT:

    TOTAL VOCAB COUNT IN TRAIN = 1164
    ---
    TOTAL TRAIN VOCAB COUNT WHOS CONTEXTUAL REF. FOUND IN POKEMON DATASET  = 520 +
    TOTAL TRAIN VOCAB COUNT WHOS CONTEXTUAL REF. FOUND IN DRAGON DATASET   = 161 +
    TOTAL TRAIN VOCAB COUNT WHOS CONTEXTUAL REF. NOT FOUND IN ABOVE DATASET= 483 = 1164
    
    
    TOTAL VOCAB COUNT IN TEST = 284
    ---
    TOTAL TRAIN VOCAB COUNT WHOS CONTEXTUAL REF. FOUND IN POKEMON DATASET  = 140 +
    TOTAL TRAIN VOCAB COUNT WHOS CONTEXTUAL REF. FOUND IN DRAGON DATASET   = 32 +
    TOTAL TRAIN VOCAB COUNT WHOS CONTEXTUAL REF. NOT FOUND IN ABOVE DATASET= 113 = 185
    ---
    185 because test_data_set_2 records contains one duplicate VOCAB "COLD" 
                    
    

In [ ]:
print(set([x for x in filter_unresolved_test_records if filter_unresolved_test_records.count(x) > 1]))
print("Count of record \'Cold 45\' appearance in the test data set:", (test_data_set_2['en'] == "Cold 45").sum())

{'Cold 45'}
Count of record 'Cold 45' appearance in the test data set: 2


Now, we will implement our approach by iterating for each English noun provided in the test/ train the data set to search them in the parallel corpus individually and save all those pairs where the search results in the success. As a result of this activity, we will create a sub-set of all the downloaded parallel corpus, where the English version text in each pair must contain at least one English noun in it.

Please note that the complete execution of the following cells will take some time depending on the configuration of the system where you are running this notebook. Because the parallel corpuses downloaded from OPUS are huge, so it will take some time to search each English noun in the English text of all the record pairs present in the corpora.

BUILDING CONTEXTUAL REFERENCE DATA FOR PROCESSING TEESTING QUERIES

In [ ]:
'''
 ## Extraction of sub-set of English/French pairs from the downloaded parallel corpus
 ## that contains atleast one English noun from the test set in it in the English version text.
'''
en = []
fr = []
store_csv_name = "c_ref_test.csv"
df_contextual_refrences = pd.DataFrame(list(zip([],[])), columns = ["en", "fr"])

## Loading POKEMON DATASET
file = open(base_path+"/dataset/JOKER/task2_pokemon_en.json")
pokemonENVocab = json.load(file)
file = open(base_path+"/dataset/JOKER/task2_pokemon_fr.json")
pokemonFrVocab = json.load(file)

## Loading DRAGON DATASET
dragon_data_set = pd.read_csv(base_path+"dataset/JOKER/dragons.csv", names=["en", "fr"])

## Loading names of downloaded corpus from the OPUS data set
processed_csv_file_names = ["opensub","books","bookshop", "wikipedia", "wiki", "ted"]
csv_file_names_head = "en-fr_"

f = np.frompyfunc(lambda x,w: w in x, 2,1)

## Extracting English/French pairs from the pokemon names
for index, row in tqdm(test_data_set_2.iterrows()):
    if(row.en in pokemonVocab):
        en.append(row.en.lower().strip())
        fr.append(pokemonFrVocab[pokemonENVocab.index(row.en)].lower().strip())

## Extracting English/French pairs from the nouns of How to train your dragon series
listed = np.array([ n.lower() for n in dragon_data_set.en])
for index, row in tqdm(test_data_set_2.iterrows()):
    word = row.en.lower()
    finds = f(listed, word)
    if(finds.sum()>0):
        en+=dragon_data_set[finds].en.str.lower().str.strip().tolist()
        fr+=dragon_data_set[finds].fr.str.lower().str.strip().tolist()

## Extracting English/French pairs from the downloaded OPUS corpus        
check_pr = [word.lower().strip() for word in test_data_set_2.en.tolist()]
for csv_file in processed_csv_file_names:
    faster_search_df = pd.read_csv(base_path+"dataset/JOKER/csv/en_processed/"+csv_file_names_head+csv_file+".csv")
    faster_search_df = faster_search_df.dropna()
    listed = faster_search_df['en'].to_numpy()
    for word in tqdm(check_pr):
        finds = f(listed, word)
        if(finds.sum()>0):
            en+=faster_search_df[finds].en.str.lower().str.strip().tolist()
            fr+=faster_search_df[finds].fr.str.lower().str.strip().tolist()

## Saving the extracted pairs            
print("c_ref_test is ready")
df_contextual_refrences.en = en
df_contextual_refrences.fr = fr
df_contextual_refrences.to_csv(base_path + "dataset/JOKER/csv/context/" + store_csv_name)


284it [00:00, 10137.38it/s]

0it [00:00, ?it/s]
284it [00:00, 1438.34it/s]

 33%|██████████████                            | 95/284 [07:57<16:12,  5.15s/it]


 68%|███████████████████████████▋             | 192/284 [16:09<07:15,  4.73s/it]


100%|█████████████████████████████████████████| 284/284 [24:11<00:00,  5.11s/it]

  5%|██▏                                       | 15/284 [00:00<00:06, 38.54it/s]


100%|█████████████████████████████████████████| 284/284 [00:06<00:00, 43.57it/s]

 15%|██████▏                                   | 42/284 [01:15<07:25,  1.84s/it]


 49%|████████████████████                     | 139/284 [04:15<04:40,  1.94s/it]


...

 53%|█████████████████████▊                   | 151/284 [00:27<00:23,  5.60it/s]


 87%|███████████████████████████████████▊     | 248/284 [00:45<00:06,  5.97it/s]


100%|█████████████████████████████████████████| 284/284 [00:51<00:00,  5.51it/s]

 42%|█████████████████▎                       | 120/284 [00:07<00:10, 15.07it/s]


100%|█████████████████████████████████████████| 284/284 [00:18<00:00, 15.60it/s]


c_ref_test is ready


In [ ]:
'''
 ## Exploring the saved sub-set of English/French pairs that contains atleast 
 ## one English noun from the test set in the English version of the text
'''
store_csv_name = "c_ref_test.csv"
df_contextual_refrences = pd.read_csv(base_path + "dataset/JOKER/csv/context/" + store_csv_name, names=["en", "fr"])
df_contextual_refrences.tail(5)

,en,fr
12743.0,"well, seven years passed, we were sold to a he...","» sept années ont passé, nous avons été vendus..."
12744.0,"in those days, nobody had bank accounts, or am...","a l'époque, personne n'avait de compte bancair..."
12745.0,"in one study, we asked college soccer players ...","pour une étude, nous avons demandé à des footb..."
12746.0,(laughter) i play sports at a four-year-old le...,et moi non plus. (rrires) mon niveau sportif e...
12747.0,"so for some reason, this was what i wanted to ...","pour une raison ou une autre, c'est ce que je ..."


Next, we will replicate the same approach for English nouns of the training data set to create a sub-set of all the downloaded parallel corpus, where the English version text in each pair must contain at least one English noun from the training data set in it. 

But before that, we will drop the record having the "official" value because previously, in the EDA, we have already seen that it contains an invalid value as its corresponding translation.

In [ ]:
'''
 ## The records having the "official" value in the English column don't have a 
 ## corresponding valid translation in French.   
'''
data_set_2[data_set_2.en == "official"]

,id,en,fr
179,NaN,official,2
214,NaN,official,1
321,NaN,official,2


In [ ]:
'''
 ## Dropping all the records where the English column has the "official" value.
'''
data_set_2 = data_set_2[data_set_2.en != "official"]

In [ ]:
data_set_2[data_set_2.en == "official"]

,id,en,fr


BUILDING CONTEXTUAL REFERENCE DATA FOR PROCESSING TRAINING QUERIES

In [ ]:
'''
 ## Extraction of sub-set of English/French pairs from the downloaded parallel corpus
 ## that contains atleast one English noun from the training set in it in the English version text.
'''
en = []
fr = []
store_csv_name = "c_ref_train.csv"
df_contextual_refrences = pd.DataFrame(list(zip([],[])), columns = ["en", "fr"])

## Loading POKEMON DATASET
file = open(base_path+"/dataset/JOKER/task2_pokemon_en.json")
pokemonENVocab = json.load(file)
file = open(base_path+"/dataset/JOKER/task2_pokemon_fr.json")
pokemonFrVocab = json.load(file)

## Loading DRAGON DATASET
dragon_data_set = pd.read_csv(base_path+"dataset/JOKER/dragons.csv", names=["en", "fr"])

## Loading names of downloaded corpus from the OPUS data set
processed_csv_file_names = ["opensub","books","bookshop", "wikipedia", "wiki", "ted"]
csv_file_names_head = "en-fr_"


f = np.frompyfunc(lambda x,w: w in x, 2,1)

## Extracting English/French pairs from the pokemon names
for index, row in tqdm(data_set_2.iterrows()):
    if(row.en in pokemonVocab):
        en.append(row.en.lower().strip())
        fr.append(pokemonFrVocab[pokemonENVocab.index(row.en)].lower().strip())

## Extracting English/French pairs from the nouns of How to train your dragon series
listed = np.array([ n.lower() for n in dragon_data_set.en])
for index, row in tqdm(data_set_2.iterrows()):
    word = row.en.lower()
    finds = f(listed, word)
    if(finds.sum()>0):
        en+=dragon_data_set[finds].en.str.lower().str.strip().tolist()
        fr+=dragon_data_set[finds].fr.str.lower().str.strip().tolist()

## Extracting English/French pairs from the downloaded OPUS corpus          
check_pr = [word.lower().strip() for word in data_set_2.en.tolist()]
for csv_file in processed_csv_file_names:
    faster_search_df = pd.read_csv(base_path+"dataset/JOKER/csv/en_processed/"+csv_file_names_head+csv_file+".csv")
    faster_search_df = faster_search_df.dropna()
    listed = faster_search_df['en'].to_numpy()
    for word in tqdm(check_pr):
        finds = f(listed, word)
        if(finds.sum()>0):
            en+=faster_search_df[finds].en.str.lower().str.strip().tolist()
            fr+=faster_search_df[finds].fr.str.lower().str.strip().tolist()

## Saving the extracted pairs            
print("c_ref_train is ready")
df_contextual_refrences.en = en
df_contextual_refrences.fr = fr
df_contextual_refrences.to_csv(base_path + "dataset/JOKER/csv/context/" + store_csv_name)


0it [00:00, ?it/s]
1161it [00:00, 10730.10it/s]

0it [00:00, ?it/s]
86it [00:00, 852.16it/s]
172it [00:00, 385.25it/s]
278it [00:00, 567.36it/s]
384it [00:00, 705.28it/s]
499it [00:00, 831.87it/s]
626it [00:00, 957.81it/s]
746it [00:00, 1026.26it/s]
889it [00:01, 1143.92it/s]
1161it [00:01, 970.94it/s] 

  8%|███                                    | 92/1161 [08:29<1:42:21,  5.75s/it]


 16%|██████▏                               | 189/1161 [17:34<1:18:34,  4.85s/it]


 25%|█████████▎                            | 286/1161 [26:32<1:28:16,  6.05s/it]


 33%|████████████▌                         | 383/1161 [35:36<1:11:48,  5.54s/it]


 41%|████████████████▌                       | 480/1161 [44:29<54:26,  4.80s/it]


...

 60%|████████████████████████▏               | 702/1161 [00:50<00:31, 14.48it/s]


 77%|██████████████████████████████▊         | 896/1161 [01:05<00:25, 10.47it/s]


 94%|████████████████████████████████████▌  | 1090/1161 [01:18<00:04, 14.47it/s]


100%|███████████████████████████████████████| 1161/1161 [01:23<00:00, 13.93it/s]


c_ref_train is ready


In [ ]:
'''
 ## Exploring the saved sub-set of English/French pairs that contains atleast 
 ## one English noun from the training set in the English version of the text
'''
store_csv_name = "c_ref_train.csv"
df_contextual_refrences = pd.read_csv(base_path + "dataset/JOKER/csv/context/" + store_csv_name, names=["en", "fr"])
df_contextual_refrences.tail(5)

,en,fr
184887.0,because while having less honey might be a buz...,"parce que si moins de miel, ce n'est pas drôle..."
184888.0,cheshire cats can juggle their own heads.,les chats du cheshire savent jongler avec leur...
184889.0,so that goes along with the cheshire cat sayin...,"« si l'endroit où tu veux aller t'importe peu,..."
184890.0,no quidditch match ends until the golden snitc...,un match de quidditch ne s'arrête que lorsque ...
184891.0,apricot halves like the ears of cherubim.,des moitiés d'abricot comme des oreilles de ch...


Next, we will count the number of English nouns from the test and training data set provided by the Joker CLEF task2 team exist in the English version text of the developed sub set of the parallel corpus from the OPUS repository.

In [ ]:
f = np.frompyfunc(lambda x,w: w in x, 2,1)

'''
 ## Count the number of English nouns from the training data set exist in the 
 ## developed context references (c_ref_train.csv) for the train data set
'''
store_csv_name = "c_ref_train.csv"
df_contextual_refrences = pd.read_csv(base_path+"dataset/JOKER/csv/context/"+store_csv_name)

check_pr = [word.lower().strip() for word in data_set_2.en.tolist()]
listed = df_contextual_refrences.en.to_numpy()

train_nf = []
count_ref_for_train_vocab = 0
for word in tqdm(check_pr):
    finds = f(listed, word)
    if (finds.sum()>0):
        count_ref_for_train_vocab +=1
    else:
        train_nf.append(word)

print("Count Of training vocabs for which contextual refrences exist in data set", count_ref_for_train_vocab, ", Other Remaining Records=", len(check_pr)-count_ref_for_train_vocab)


'''
 ## Count the number of English nouns from the test data set exist in the 
 ## developed context references (c_ref_test.csv) for the test data set
'''
store_csv_name = "c_ref_test.csv"
df_contextual_refrences = pd.read_csv(base_path+"dataset/JOKER/csv/context/"+store_csv_name)

check_pr = [word.lower().strip() for word in test_data_set_2.en.tolist()]
listed = df_contextual_refrences.en.to_numpy()

test_nf = []
count_ref_for_test_vocab=0
for word in tqdm(check_pr):
    finds = f(listed, word)
    if (finds.sum()>0):
        count_ref_for_test_vocab +=1
    else:
        test_nf.append(word)
        
print("Count Of testing vocabs for which contextual refrences exist in data set", count_ref_for_test_vocab, ", Other Remaining Records=", len(check_pr)-count_ref_for_test_vocab)



 25%|█████████▉                              | 288/1161 [00:12<00:35, 24.28it/s]


 50%|███████████████████▉                    | 579/1161 [00:24<00:24, 23.68it/s]


 75%|█████████████████████████████▉          | 870/1161 [00:36<00:12, 23.80it/s]


100%|███████████████████████████████████████| 1161/1161 [00:48<00:00, 23.96it/s]


Count Of training vocabs for which contextual refrences exist in data set 874 , Other Remaining Records= 287



100%|████████████████████████████████████████| 284/284 [00:00<00:00, 390.40it/s]

Count Of testing vocabs for which contextual refrences exist in data set 219 , Other Remaining Records= 65


The contextual references created for the English nouns from the training data set provided by JOKER CLEF task 2 team. It is important to ensure that those contextual reference pairs must contain atleast one English noun in the English version of the text and its corresponding translation in the French version of the text. Previously  we have ensured that they must have atleast one English noun in it, and now we will code another iteratation to make sure that they also have corresponding French translation in the French version of the text. Plus, now we will also change the structure of the saved contextual references from the two columns to classical Extractive Q/A style.

In [ ]:
f = np.frompyfunc(lambda x,w: w in x.split(), 2,1)
train_dataset_QA = pd.DataFrame(list(zip([],[], [], [], [])), columns = ['id', 'context', 'question', 'answers'])

store_csv_name = "c_ref_train.csv"
df_contextual_refrences = pd.read_csv(base_path+"dataset/JOKER/csv/context/"+store_csv_name)

check_pr_en = [word.lower().strip() for word in data_set_2.en.tolist()]
chcek_pr_fr = [word.lower().strip() for word in data_set_2.fr.tolist()]

listed_en = df_contextual_refrences.en.to_numpy()
listed_fr = df_contextual_refrences.fr.to_numpy()

train_nf = []
count_ref_for_train_vocab = 0

ids = []
questions = []
contexts = []
answers = []

for word_en, word_fr in tqdm(zip(check_pr_en, chcek_pr_fr)):
    
    finds_en = f(listed_en, word_en.strip())
    finds_fr = f(listed_fr, word_fr.strip())
    
    if (finds_en.sum()>0 and finds_fr.sum()>0):
        count_ref_for_train_vocab +=1
        french_contexts = df_contextual_refrences[finds_fr].fr.to_list()
        questions+=[word_en for _ in french_contexts]
        contexts+=french_contexts
        for context_fr in french_contexts:
            answers.append(json.dumps({"text": [word_fr], "answer_start": [context_fr.find(word_fr)]}))
    else:
        train_nf.append(word)

train_dataset_QA.id = list(range(0,len(questions)))
train_dataset_QA.question = questions
train_dataset_QA.context = contexts
train_dataset_QA.answers = answers

train_dataset_QA.to_csv(base_path +"dataset/JOKER/Task 2/train/joker_task2_train_context_aware_data_where_answers_exist.csv")
print("Count Of training vocabs for which contextual refrences exist in data set", count_ref_for_train_vocab, ", Other Remaining Records=", len(check_pr_en)-count_ref_for_train_vocab)



0it [00:00, ?it/s]
1it [00:00,  1.07it/s]
2it [00:01,  1.11it/s]
3it [00:02,  1.15it/s]
4it [00:03,  1.15it/s]
5it [00:04,  1.11it/s]
6it [00:05,  1.10it/s]
7it [00:06,  1.13it/s]
8it [00:07,  1.12it/s]
9it [00:08,  1.09it/s]
10it [00:09,  1.10it/s]
11it [00:09,  1.11it/s]
12it [00:10,  1.10it/s]
13it [00:11,  1.09it/s]
14it [00:12,  1.09it/s]
15it [00:13,  1.08it/s]
16it [00:14,  1.10it/s]
17it [00:15,  1.10it/s]
18it [00:16,  1.12it/s]
19it [00:17,  1.16it/s]
20it [00:17,  1.17it/s]
21it [00:18,  1.19it/s]
22it [00:19,  1.19it/s]
23it [00:20,  1.20it/s]
24it [00:21,  1.17it/s]
25it [00:22,  1.17it/s]
26it [00:22,  1.17it/s]
27it [00:23,  1.13it/s]
28it [00:24,  1.12it/s]
29it [00:25,  1.10it/s]
30it [00:26,  1.10it/s]
31it [00:27,  1.12it/s]
32it [00:28,  1.14it/s]
33it [00:29,  1.14it/s]
34it [00:30,  1.14it/s]
35it [00:30,  1.15it/s]
36it [00:31,  1.16it/s]
37it [00:32,  1.16it/s]
38it [00:33,  1.12it/s]
39it [00:34,  1.12it/s]
40it [00:35,  1.13it/s]
41it [00:36,  1.13it/s]
42it 

296it [04:29,  1.03s/it]
297it [04:30,  1.07s/it]
298it [04:32,  1.07s/it]
299it [04:32,  1.04s/it]
300it [04:33,  1.02s/it]
301it [04:34,  1.02s/it]
302it [04:35,  1.01s/it]
303it [04:36,  1.01s/it]
304it [04:37,  1.01s/it]
305it [04:38,  1.01it/s]
306it [04:39,  1.03it/s]
307it [04:40,  1.04it/s]
308it [04:41,  1.07it/s]
309it [04:42,  1.08it/s]
310it [04:43,  1.05it/s]
311it [04:44,  1.02it/s]
312it [04:45,  1.05it/s]
313it [04:46,  1.07it/s]
314it [04:47,  1.09it/s]
315it [04:48,  1.10it/s]
316it [04:49,  1.11it/s]
317it [04:49,  1.12it/s]
318it [04:50,  1.13it/s]
319it [04:51,  1.12it/s]
320it [04:52,  1.12it/s]
321it [04:53,  1.12it/s]
322it [04:54,  1.12it/s]
323it [04:55,  1.12it/s]
324it [04:56,  1.11it/s]
325it [04:57,  1.13it/s]
326it [04:57,  1.10it/s]
327it [04:58,  1.12it/s]
328it [04:59,  1.13it/s]
329it [05:00,  1.13it/s]
330it [05:01,  1.14it/s]
331it [05:02,  1.15it/s]
332it [05:03,  1.11it/s]
333it [05:04,  1.12it/s]
334it [05:05,  1.13it/s]
335it [05:05,  1.13it/s]


588it [08:47,  1.13it/s]
589it [08:48,  1.14it/s]
590it [08:49,  1.14it/s]
591it [08:50,  1.11it/s]
592it [08:51,  1.12it/s]
593it [08:52,  1.12it/s]
594it [08:53,  1.10it/s]
595it [08:54,  1.12it/s]
596it [08:55,  1.12it/s]
597it [08:55,  1.13it/s]
598it [08:56,  1.14it/s]
599it [08:57,  1.15it/s]
600it [08:58,  1.15it/s]
601it [08:59,  1.15it/s]
602it [09:00,  1.15it/s]
603it [09:01,  1.15it/s]
604it [09:02,  1.15it/s]
605it [09:02,  1.15it/s]
606it [09:03,  1.16it/s]
607it [09:04,  1.15it/s]
608it [09:05,  1.15it/s]
609it [09:06,  1.15it/s]
610it [09:07,  1.12it/s]
611it [09:08,  1.12it/s]
612it [09:09,  1.14it/s]
613it [09:09,  1.14it/s]
614it [09:10,  1.12it/s]
615it [09:11,  1.13it/s]
616it [09:12,  1.13it/s]
617it [09:13,  1.12it/s]
618it [09:14,  1.10it/s]
619it [09:15,  1.09it/s]
620it [09:16,  1.10it/s]
621it [09:17,  1.09it/s]
622it [09:18,  1.03it/s]
623it [09:19,  1.04it/s]
624it [09:20,  1.05it/s]
625it [09:21,  1.07it/s]
626it [09:22,  1.09it/s]
627it [09:22,  1.10it/s]


880it [12:55,  1.19it/s]
881it [12:56,  1.20it/s]
882it [12:57,  1.20it/s]
883it [12:57,  1.20it/s]
884it [12:58,  1.20it/s]
885it [12:59,  1.20it/s]
886it [13:00,  1.19it/s]
887it [13:01,  1.18it/s]
888it [13:02,  1.18it/s]
889it [13:02,  1.18it/s]
890it [13:03,  1.19it/s]
891it [13:04,  1.19it/s]
892it [13:05,  1.20it/s]
893it [13:06,  1.19it/s]
894it [13:07,  1.19it/s]
895it [13:07,  1.18it/s]
896it [13:08,  1.18it/s]
897it [13:09,  1.19it/s]
898it [13:10,  1.19it/s]
899it [13:11,  1.19it/s]
900it [13:12,  1.19it/s]
901it [13:13,  1.19it/s]
902it [13:13,  1.18it/s]
903it [13:14,  1.17it/s]
904it [13:15,  1.18it/s]
905it [13:16,  1.19it/s]
906it [13:17,  1.20it/s]
907it [13:18,  1.21it/s]
908it [13:18,  1.21it/s]
909it [13:19,  1.22it/s]
910it [13:20,  1.22it/s]
911it [13:21,  1.22it/s]
912it [13:22,  1.22it/s]
913it [13:22,  1.21it/s]
914it [13:23,  1.21it/s]
915it [13:24,  1.20it/s]
916it [13:25,  1.21it/s]
917it [13:26,  1.22it/s]
918it [13:27,  1.22it/s]
919it [13:27,  1.22it/s]


Count Of training vocabs for which contextual refrences exist in data set 669 , Other Remaining Records= 492


In [ ]:
'''
 ## Exploring the saved Q/A style contextual reference for each English noun in the training data set.
'''
train_dataset_path = "/dataset/JOKER/Task 2/train/joker_task2_train_context_aware_data_where_answers_exist.csv"
data_set_with_context_ref = pd.read_csv(base_path+train_dataset_path)
print("Total number of Q/A records:", len(data_set_with_context_ref))
data_set_with_context_ref.head(5)

Total number of Q/A records: 2172


,Unnamed: 0,id,context,question,answers
0,0,0,capidextre,ambipom,"{""text"": [""capidextre""], ""answer_start"": [0]}"
1,1,1,efflèche,dartrix,"{""text"": [""effl\u00e8che""], ""answer_start"": [0]}"
2,2,2,sepiatroce,malamar,"{""text"": [""sepiatroce""], ""answer_start"": [0]}"
3,3,3,croquine,bounsweet,"{""text"": [""croquine""], ""answer_start"": [0]}"
4,4,4,"eh, astérix et obélix sont de retour.",obelix,"{""text"": [""ob\u00e9lix""], ""answer_start"": [15]}"


We will do a simillar kind of transformation for the contextual references created for the English nouns that belongs from the test data set provided by Joker CLEF task2 team. We will change its structure from two columns to classical extractive Q/A data set style. Its important to note that, as we don't have the corresponding French translations for the English nouns of test data set. So, here there is no need to further filter data set to check their French version of the text contains the corresponding translation or not. Here, we will assume that they must contain the corrsponding translation for atleast one English noun in them.

In [ ]:
f = np.frompyfunc(lambda x,w: w in x.split(), 2,1)
test_dataset_QA = pd.DataFrame(list(zip([],[], [], [], [])), columns = ['id', 'context', 'question'])

store_csv_name = "c_ref_test.csv"
df_contextual_refrences = pd.read_csv(base_path+"dataset/JOKER/csv/context/"+store_csv_name)

check_pr_en = [word.lower().strip() for word in test_data_set_2.en.tolist()]

listed_en = df_contextual_refrences.en.to_numpy()

test_nf = []
count_ref_for_test_vocab = 0

ids = []
questions = []
contexts = []
answers = []

for word_en in tqdm(check_pr_en):
    
    finds_en = f(listed_en, word_en.strip())
    
    if (finds_en.sum()>0):
        count_ref_for_test_vocab +=1
        french_contexts = df_contextual_refrences[finds_en].fr.to_list()
        questions+=[word_en for _ in french_contexts]
        contexts+=french_contexts
    
    else:
        test_nf.append(word)

test_dataset_QA.id = list(range(0,len(questions)))
test_dataset_QA.question = questions
test_dataset_QA.context = contexts

test_dataset_QA.to_csv(base_path +"dataset/JOKER/Task 2/test/joker_task2_test_context_aware_data_where_answers_exist.csv")
print("Count Of testing vocabs for which contextual refrences exist in data set", count_ref_for_test_vocab, ", Other Remaining Records=", len(check_pr_en)-count_ref_for_test_vocab)



100%|█████████████████████████████████████████| 284/284 [00:07<00:00, 37.50it/s]

Count Of testing vocabs for which contextual refrences exist in data set 185 , Other Remaining Records= 99


In [ ]:
'''
 ## Exploring the saved Q/A style contextual reference for each English noun in the test data set.
'''
train_dataset_path = "/dataset/JOKER/Task 2/test/joker_task2_test_context_aware_data_where_answers_exist.csv"
data_set_with_context_ref = pd.read_csv(base_path+train_dataset_path)
print("Total number of Q/A records:", len(data_set_with_context_ref))
data_set_with_context_ref.head(5)

Total number of Q/A records: 6176


,Unnamed: 0,id,context,question
0,0,0,astronelle,orbeetle
1,1,1,tournicoton,gossifleur
2,2,2,blancoton,eldegoss
3,3,3,crabominable,crabominable
4,4,4,rubombelle,ribombee
